## 2-2、三种计算图
有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph。  

在Tensorflow1.0时代，采用的是静态计算图，需要先使用Tensorflow的各种算子创建计算图，然后再开启一个会话，显式执行计算图。   

而在2.0时代，采用的是动态计算图，即计算每一个算子后，该算子会被动态加入到隐含的默认计算图中立即执行得到结果，而无需开启session。  

使用动态计算图的好处是方便调试程序，它会让tensorflow代码的表现和python原生代码的表现一样，写起来就写numpy一样，各种日志打印，控制流全部都是可以使用的。  

使用动态计算图的缺点是运行效率会相对低一些。因为会有许多次python进程和c++进程之间的通信。而静态计算图构建完成之后几乎全部在tensorflow内核上使用c++代码执行，效率更高。此外静态图会对计算步骤进行一定的优化，剪去和结果无关的计算步骤。   

如果需要在tensorflow2.0中使用静态图，可以使用@tf.function装饰器将普通python函数转换成对应的tensorflow计算图构建代码。运行该函数就相当于在tensorflow1.0中用session执行代码，使用tf.function构建静态图的方式叫做Autograph.   

### 一、计算图简介
计算图由节点(nodes)和线(edges)组成。  

节点表示操作符Operator，或者称之为算子，线表示计算间的依赖。

实线表示有数据传递依赖，传递的数据即张量。

虚线通常可以表示控制依赖，即执行先后顺序。  

![]("data/strjoin_graph.png")  


### 二、静态计算图
在tensorflow1.0中，使用静态计算图分为两步，第一步定义计算图，第二步在会话中执行计算图。  

#### Tensorflow1.0 静态计算图范例(没装1.0，无法运行)

In [4]:
import tensorflow as tf

g = tf.Graph()
with g.as_default():
    x = tf.placeholder(name = 'x',shape=[],dtype=tf.string)
    y = tf.placeholder(name = 'y',shape=[],dtype=tf.string)
    z = tf.string_join([x,y],name = 'join',separator='')
with tf.Session(graph=g) as sess:
    print(sess.run(fetches=z,feed_dict={x:"hello",y:"world"}))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

#### Tensorflow2.0怀旧版静态计算图
Tensorflow2.0为了确保对老版本tensorflow项目的兼容性，在tf.compat.v1子模块中保留了对Tensorflow1.0那种静态计算图构建风格的支持。  
可称之为怀旧版静态计算图，已经不推荐使用了。

In [8]:
import tensorflow as tf

g = tf.compat.v1.Graph()
with g.as_default():
    x = tf.compat.v1.placeholder(name = 'x',shape=[],dtype=tf.string)
    y = tf.compat.v1.placeholder(name = 'y',shape=[],dtype=tf.string)
    z = tf.strings.join([x,y],name = 'join',separator='')
with tf.compat.v1.Session(graph=g) as sess:
    print(sess.run(fetches=z,feed_dict={x:"hello",y:"world"}))

b'helloworld'


### 三、动态计算图
在tensorflow2.0中，使用的是动态计算图和Autograph。  

在1.0中，使用静态计算图分为两步，第一步定义计算图，第二步在会话中执行计算图。  

动态计算图已经不区分计算图的定义和执行了，而是定义后立即执行。因此称之为Eager Excution.  

Eager这个应为单词的愿意是“迫不及待的”，也就是立即执行的意思。  

In [9]:
x = tf.constant("hello")
y = tf.constant("world")
z = tf.strings.join([x,y],separator=" ")
tf.print(z)

hello world


In [10]:
# 可以将动态计算图代码的输入和输出关系封装成函数

def strjoin(x,y):
    z =  tf.strings.join([x,y],separator = " ")
    tf.print(z)
    return z

result = strjoin(tf.constant("hello"),tf.constant("world"))
print(result)

hello world
tf.Tensor(b'hello world', shape=(), dtype=string)


### 四、Tensorflow2.0的Autograph
动态计算图运行效率比较低。   

可以使用@tf.function装饰器将普通python函数转换成和tensorflow1.0中对应的静态计算图构建代码。  

在TensorFlow1.0中，使用计算图分两步，第一步定义计算图，第二步在会话中执行计算图。

在TensorFlow2.0中，如果采用Autograph的方式使用计算图，第一步定义计算图变成了定义函数，第二步执行计算图变成了调用函数。

不需要使用会话了，一些都像原始的Python语法一样自然。

实践中，我们一般会先用动态计算图调试代码，然后在需要提高性能的的地方利用@tf.function切换成Autograph获得更高的效率。

当然，@tf.function的使用需要遵循一定的规范，我们后面章节将重点介绍。

In [11]:
import tensorflow as tf

@tf.function
def strjoin(x,y):
    z = tf.strings.join([x,y],separator=" ")
    tf.print(z)
    return z

result = strjoin(tf.constant("Hi"),tf.constant("Mary"))
print(result)

Hi Mary
tf.Tensor(b'Hi Mary', shape=(), dtype=string)


In [13]:
import datetime

stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = './data/autograph/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

tf.summary.trace_on(graph=True,profiler=True)

result = strjoin("hello","world")

#将计算图信息写入日志
with writer.as_default():
    tf.summary.trace_export(
        name="autograph",
        step=0,
        profiler_outdir=logdir)

hello world


In [16]:
#启动 tensorboard在jupyter中的魔法命令
import os
PATH = os.getenv("PATH")
%env PATH = /home/tangxi.zq/.conda/envs/tf2/bin:$PATH
%load_ext tensorboard

env: PATH=/home/tangxi.zq/.conda/envs/tf2/bin:/usr/local/bin:/home/tangxi.zq/.conda/envs/torch/bin:/home/tangxi.zq/tangxi/anaconda3/condabin:/sbin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/usr/X11R6/bin:/home/tangxi.zq/.local/bin:/home/tangxi.zq/bin:/home/tangxi.zq/tangxi/anaconda3/bin:/home/tangxi.zq/bin
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [17]:
#启动tensorboard
%tensorboard --logdir ./data/autograph/